<a href="https://colab.research.google.com/github/b4dr0s1992/alizul01/blob/main/Object_Detection_2023_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/khanhlvg/tflite_raspberry_pi/blob/main/object_detection/Train_custom_model_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Train a custom object detection model with MediaPipe Model Maker

In this colab notebook, you'll learn how to use [MediaPipe Model Maker](https://developers.google.com/mediapipe/solutions/model_maker) to train a custom object detection model.

The Model Maker library uses *transfer learning* to simplify the process of training a TensorFlow Lite model using a custom dataset to use with the [MediaPipe Object Detector task](https://developers.google.com/mediapipe/solutions/vision/object_detector).


## Preparation

### Install the required packages
Start by installing the required packages, including the Model Maker package from the [GitHub repo](https://github.com/google/mediapipe/tree/master/mediapipe/model_maker) and the pycocotools library you'll use for evaluation.

In [4]:
!python --version

Python 3.12.12


In [5]:
!pip install --upgrade pip

In [8]:
!pip install https://github.com/google-ai-edge/mediapipe/tree/master/mediapipe/model_maker

     - 186.9 kB 6.3 MB/s 0:00:00
  ERROR: Cannot unpack file /tmp/pip-unpack-4m4d7tse/model_maker (downloaded from /tmp/pip-req-build-ajs4uoa2, content-type: text/html; charset=utf-8); cannot detect archive format
ERROR: Cannot determine archive format of /tmp/pip-req-build-ajs4uoa2


In [9]:
!pip install mediapipe-model-maker

  Using cached mediapipe_model_maker-0.2.1.4-py3-none-any.whl.metadata (1.7 kB)
  Using cached mediapipe-0.10.21-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (9.7 kB)
INFO: pip is looking at multiple versions of mediapipe-model-maker to determine which version is compatible with other requirements. This could take a while.
  Using cached mediapipe_model_maker-0.2.1.3-py3-none-any.whl.metadata (1.6 kB)
  Using cached mediapipe_model_maker-0.2.1.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached mediapipe_model_maker-0.2.1.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached mediapipe_model_maker-0.2.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached mediapipe_model_maker-0.2.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached tf_models_official-2.11.6-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached PyYAML-5.4.1.tar.gz (175 kB)
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ e

Import the required packages.

In [7]:
from google.colab import files
import os
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import image_classifier

import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'mediapipe_model_maker'

### Prepare the dataset



In [ ]:
!unzip fruits.zip
train_dataset_path = "fruits/train"
validation_dataset_path = "fruits/valid"
test_dataset_path = "fruits/test"

Archive:  fruits.zip
   creating: fruits/
  inflating: fruits/README.dataset.txt  
  inflating: fruits/README.roboflow.txt  
   creating: fruits/test/
   creating: fruits/test/Annotations/
  inflating: fruits/test/Annotations/Image_17_jpg.rf.fcd26bb58f5144b4b9c78a2f4eedd6dd.xml  
  inflating: fruits/test/Annotations/Image_1_jpg.rf.003d5f563b2a4060b0e0f696d04651bd.xml  
  inflating: fruits/test/Annotations/Image_1_jpg.rf.14c2cca49426ef3600a5c044cf8ba14a.xml  
  inflating: fruits/test/Annotations/Image_1_jpg.rf.7ea0fb5ba2e69236d3854fe867cc84d6.xml  
  inflating: fruits/test/Annotations/Image_1_jpg.rf.808f03c66ec40e0ba676831d4421df9d.xml  
  inflating: fruits/test/Annotations/Image_1_jpg.rf.9306b20c9ce0b1f73861f678cee3d757.xml  
  inflating: fruits/test/Annotations/Image_1_jpg.rf.962fa47c1862633dbdc9b8844ece5b43.xml  
  inflating: fruits/test/Annotations/Image_1_jpg.rf.9f2049b64428336627b6baa8b0c46725.xml  
  inflating: fruits/test/Annotations/Image_1_jpg.rf.a3c87d06367fa7d7f63637f2b858ec

## Train the object detection model

### Step 1: Load the dataset

* Images in `train_data` is used to train the custom object detection model.
* Images in `val_data` is used to check if the model can generalize well to new images that it hasn't seen before.

In [ ]:
train_data = object_detector.Dataset.from_pascal_voc_folder(
    'fruits/train',
    cache_dir="/tmp/od_data/train",
)

val_data = object_detector.Dataset.from_pascal_voc_folder(
    'fruits/valid',
    cache_dir="/tmp/od_data/validatation")

test_data = object_detector.Dataset.from_pascal_voc_folder(
    'fruits/test',
    cache_dir="/tmp/od_data/test")

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


### Step 2: Train the TensorFlow model with the training data.

* Set `batch_size = 8` here so you will see that it takes 13 steps to go through the 139 images in the training dataset.
* Set `learning_rate = 0.3` here so the model will adjust its weights at a rate of 0.3.
* Set `epochs = 50`, which means it will go through the training dataset 50 times. You can look at the validation accuracy during training and stop when you see validation loss (`val_loss`) stop decreasing to avoid overfitting.


In [ ]:
hparams = object_detector.HParams(batch_size=8, learning_rate=0.3, epochs=50, export_dir='exported_model')
options = object_detector.ObjectDetectorOptions(
    supported_model=object_detector.SupportedModels.MOBILENET_V2,
    hparams=hparams
)
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=val_data,
    options=options)

/usr/local/lib/python3.10/dist-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


Model: "retina_net_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net (MobileNet)      {'2': (None, 64, 64, 24)  2257984   
                             , '3': (None, 32, 32, 32            
                             ),                                  
                              '4': (None, 16, 16, 96)            
                             , '5': (None, 8, 8, 320)            
                             , '6': (None, 8, 8, 1280            
                             )}                                  
                                                                 
 fpn (FPN)                   {'5': (None, 8, 8, 128),  149056    
                              '4': (None, 16, 16, 128            
                             ),                                  
                              '3': (None, 32, 32, 128            
                             ),                   

/usr/local/lib/python3.10/dist-packages/keras/backend.py:451: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


62/62 [==============================] - 83s 398ms/step - total_loss: 2.0072 - cls_loss: 1.6366 - box_loss: 0.0063 - model_loss: 1.9513 - val_total_loss: 1.4107 - val_cls_loss: 0.9781 - val_box_loss: 0.0075 - val_model_loss: 1.3544
Epoch 2/50
62/62 [==============================] - 17s 282ms/step - total_loss: 1.0296 - cls_loss: 0.7597 - box_loss: 0.0043 - model_loss: 0.9732 - val_total_loss: 0.8680 - val_cls_loss: 0.6175 - val_box_loss: 0.0039 - val_model_loss: 0.8117
Epoch 3/50
62/62 [==============================] - 18s 283ms/step - total_loss: 0.7783 - cls_loss: 0.5655 - box_loss: 0.0031 - model_loss: 0.7220 - val_total_loss: 0.7699 - val_cls_loss: 0.5322 - val_box_loss: 0.0036 - val_model_loss: 0.7136
Epoch 4/50
62/62 [==============================] - 18s 289ms/step - total_loss: 0.6557 - cls_loss: 0.4674 - box_loss: 0.0026 - model_loss: 0.5993 - val_total_loss: 0.7340 - val_cls_loss: 0.4940 - val_box_loss: 0.0037 - val_model_loss: 0.6777
Epoch 5/50
62/62 [=====================

### Step 3. Evaluate the model with the validation data.

After training the object detection model using the images in the training dataset, use the images in the validation dataset to evaluate how the model performs against new data it has never seen before.

The evaluation metrics are same as [COCO](https://cocodataset.org/#detection-eval).

In [ ]:
loss, coco_metrics = model.evaluate(test_data, batch_size=4)
print(f"Validation loss: {loss}")
print(f"Validation coco metrics: {coco_metrics}")

15/15 [==============================] - 2s 79ms/step - total_loss: 0.7866 - cls_loss: 0.5750 - box_loss: 0.0031 - model_loss: 0.7303
creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.29s).
Accumulating evaluation results...
DONE (t=0.07s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.440
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.664
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.505
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.440
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.254
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.587
 Average Recall     (AR) @[

### Step 4: Export as a TensorFlow Lite model.

After creating the model, convert and export it to a Tensorflow Lite model format, and then download it for later use in an on-device application with MediaPipe Tasks.

In [ ]:
model.export_model('fruits.tflite')
!ls exported_model
files.download('exported_model/fruits.tflite')

Exporting a floating point model


/usr/local/lib/python3.10/dist-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


float_ckpt.data-00000-of-00001	float_ckpt.index  fruits.tflite  metadata.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>